In [2]:
library(tidyverse)
library(data.table)
library(magrittr)
setwd("~/museum_data/rawdata/")

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.0     ✔ stringr 1.3.0
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose


Attaching package: ‘magrittr’

The following object is masked from ‘package:purrr’:

    set_names

The following object is masked from ‘package:tidyr’:

    extract



## VERTNET

In [82]:
## Neither fread nor read.table read in the right number of rows, this is the only way
tmp<-matrix(ncol=174,nrow=150000) 
for (i in 1:150000) {
    row = unlist(strsplit(x[i+1],split="\t"))
    if (length(row) < 174) {print(i)}
    tmp[i,]<- row
    #
}
colnames(tmp)<-unlist(strsplit(x[1],split="\t")) 
vertnet <-data.frame(tmp,stringsAsFactors=F) 

## Create ids ##
vertnet %<>% mutate(ids = paste(institutioncode,catalognumber,sep=":"))

## Select relevant columns. This will become the template for all subsequent datasets ##
vertnet %<>% dplyr::select(ids,
                          scientificname,
                          sex,
                          stateprovince,
                          locality,
                          decimallongitude,
                          decimallatitude,
                          preparations,
                          lifestage,
                          hastissue,
                          dynamicproperties,
                          habitat,
                          year)

head(vertnet,1)

ids,scientificname,sex,stateprovince,locality,decimallongitude,decimallatitude,preparations,lifestage,hastissue,dynamicproperties,habitat,year
CAS:1001,Peromyscus maniculatus sonoriensis,male,California,Buttonwillow,-119.46583,35.40019,SS,,0,,,1914


## Arctos

In [95]:
arctos <- read.csv("arctos_pmanic_full.csv") # fread was throwing annoying erros, this read.csv doesn't take too long

In [96]:
## Select same columns as vertnet, plus a bit more 
arctos %<>% dplyr::select(GUID,
                          SPECIES,
                          SEX,
                          STATE_PROV,
                          SPEC_LOCALITY,
                          DEC_LONG,
                          DEC_LAT,
                          PARTS,
                          AGE_CLASS,
                          HAS_TISSUES,
                          HABITAT,
                          YEAR,
                          # other fields that would be under 'attributes' in VERTNET
                          TOTAL_LENGTH,
                          TAIL_LENGTH,
                          HIND_FOOT_WITH_CLAW,
                          EAR_FROM_NOTCH,
                          WEIGHT) %>%
    set_colnames(c("ids","scientificname","sex","stateprovince","locality","decimallongitude","decimallatitude",
                   "preparations","lifestage","hastissue","habitat","year",
                   # Morphological traits
                   "total","tail","hind_foot","ear","weight")) %>%
    mutate(dynamicproperties=NA,ids=gsub("*:(\\w+):*",":",ids))

head(arctos,1)

ids,scientificname,sex,stateprovince,locality,decimallongitude,decimallatitude,preparations,lifestage,hastissue,habitat,year,total,tail,hind_foot,ear,weight,dynamicproperties
UAM:51334,Peromyscus maniculatus,female,Colorado,,-106.8333,37.03333,"skin; skull; skeleton, postcranial",,0,,1996,158 mm,64 mm,20 mm,20 mm,,NA


## Find redundant IDs between VERTNET and ARCTOS

Let's start by seeing which institutions are uniquely found in VERTNET or ARCTOS

In [123]:
vertnet_codes = vertnet$ids %>% str_split(":")
arctos_codes = arctos$ids %>% str_split(":")
print("Unique to vertnet")
unique(vertnet_codes %>% map(1))[!unique(vertnet_codes %>% map(1)) %in% unique(arctos_codes %>% map(1))]
print("Unique to arctos")
unique(arctos_codes %>% map(1))[!unique(arctos_codes %>% map(1)) %in% unique(vertnet_codes %>% map(1))]

[1] "Unique to vertnet"


[[1]]
[1] "CAS"

[[2]]
[1] "CRCM"

[[3]]
[1] "LACM"

[[4]]
[1] "MCZ"

[[5]]
[1] "MSU"

[[6]]
[1] "NCSM"

[[7]]
[1] "NYSM"

[[8]]
[1] "OMNH"

[[9]]
[1] "PSM"

[[10]]
[1] "QM"

[[11]]
[1] "ROM"

[[12]]
[1] "TTU"

[[13]]
[1] "University of Alberta Museums"

[[14]]
[1] "UBCBBM"

[[15]]
[1] "UMMZ"

[[16]]
[1] "YPM"

[[17]]
[1] "CM"

[[18]]
[1] "KU"

[[19]]
[1] "AMNH"

[[20]]
[1] "ASNHC"

[[21]]
[1] "BPBM"

[[22]]
[1] "CMN"

[[23]]
[1] "CSULB"

[[24]]
[1] "FHSM"

[[25]]
[1] "FMNH"

[[26]]
[1] "HSU"

[[27]]
[1] "ISU"

[[28]]
[1] "KSTC"

[[29]]
[1] "LSUMZ"

[[30]]
[1] "MHNG"

[[31]]
[1] "NMMNH"

[[32]]
[1] "\"Ohio State University - Mammal Division, Columbus, OH (OSUM)\""

[[33]]
[1] "SBMNH"

[[34]]
[1] "SDNHM"

[[35]]
[1] "TCWC"

[[36]]
[1] "TTRS"

[[37]]
[1] "UAZ"

[[38]]
[1] "UCLA"

[[39]]
[1] "UF"

[[40]]
[1] "USNM"

[[41]]
[1] "PBDB"

[1] "Unique to arctos"


[[1]]
[1] "MVZObs"

[[2]]
[1] "DGR"

[[3]]
[1] "UNR"

[[4]]
[1] "COA"

Now let's combine the datasets, retaining only ARCTOS entries that ARE NOT in vertnet

In [126]:
full_dataset = rbind(vertnet %>%
                     mutate(total=NA,tail=NA,hind_foot=NA,ear=NA,weight=NA),
                     arctos %>% 
                     dplyr::filter(!ids %in% vertnet$ids))

## Get morphological data from attributes field

In [139]:
## First define a few useful functions ##

numextract <- function(string){ 
  str_extract(string, "\\-*\\d+\\.*\\d*")
} 

# Gets first number appearing after pattern, given input string
get_following_number = function(string,pattern){
    expr = paste0("(?<=",pattern,").*")
    val = numextract(str_extract(string=string, pattern=regex(expr,ignore_case = TRUE)))[1]
    return(val)
}


rm_weird_tailvals <- function(df){
   df %>% filter(!tail_body_ratio < 0.4) %>% filter (!tail_body_ratio > 1.4)
}

## Now let's extract data for the different types of formatting we'll see in vertnet ##

## 1. '"{""measurements"":""158-63-21-19"",""weightInGrams"":""19.0"" }"'

## 2. 'sex=male ; total length=143 mm; tail length=68 mm; hind foot with claw=17 mm; ear from notch=16 mm; weight=11.5 g'

## 3. '"{""totalLengthInmm"":""183"", ""tailLengthInmm"":""99"", ""hindfootLengthInmm"":""23"", ""earLengthInmm"":""15"", ""weightIn"":""X""}"'

## 4. '"{""earLengthInMM"":""16"", ""hindfootLengthInMM"":""20"", ""tail"":""65"", ""totalLength"":""143"", ""weight"":""13.9"" }"'

In [156]:
h = grep("measurements",full_dataset$dynamicproperties)
full_dataset$dynamicproperties[h[sample(1:length(h),10)]]

[1] "\"{\"\"measurements\"\":\"\"165-71-19-18=19 g\"\" }\""                                                                                                                                                  
 [2] "unformatted measurements=155-65-21-19=20 g ; sex=male ; reproductive data=Testes not descended"                                                                                                         
 [3] "\"{\"\"measurements\"\":\"\"145-60-21-17\"\" }\""                                                                                                                                                       
 [4] "sex=male ; unformatted measurements=170-75-21-20=18 ; SNV results=yes ; verbatim collector=Hantavirus Johns Hopkins University"                                                                         
 [5] "sex=male ; reproductive data=TD=11x7 ; unformatted measurements=Verbatim weight=24;ToL=166;TaL=73;HF=21;E=21; ; total length=166 mm; tail length=73 mm; hind foot with claw=21 mm; ear from notch=21 mm"
 [6] "\"{\"\"measurements\"\":\"\"144-71-19-18\"\",\"\"weightInGrams\"\":\"\"21.3\"\" }\""                                                                                                                    
 [7] "unformatted measurements=161-69-19-16=32 g ; sex=male ; reproductive data=Testes descended - 6x2 mm"                                                                                                    
 [8] "\"{\"\"measurements\"\":\"\"150-70-19-17\"\" }\""                                                                                                                                                       
 [9] "\"verbatim collector=J.J. Mooney ; sex=unknown ; unformatted measurements=L: 148, T: 65, HF: 19 ; total length=148 mm; tail length=65 mm; hind foot with claw=19 mm\""                                  
[10] "\"{\"\"measurements\"\":\"\"150-65-20-17\"\" }\""

In [163]:
test='"{""measurements"":""165-71-19-18=19 g"" }"'

str_extract(test,regex(,ignore_case=TRUE))

[1] "165"

[1] "165"

In [130]:
full_dataset$dynamicproperties[sample(1:nrow(full_dataset),10)]

[1] "sex=female"                                                                                                                                                                                                                         
 [2] "ear from notch=23 mm; hind foot with claw=26 mm; tail length=95 mm; total length=190 mm; reproductive data=Emb. nodules 6 mm ; verbatim collector=R. L. & V. R. Rausch ; verbatim collector=R. L. & V. R. Rausch 49385 ; sex=female"
 [3] "sex=male"                                                                                                                                                                                                                           
 [4] ""                                                                                                                                                                                                                                   
 [5] NA                                                                                                                                                                                                                                   
 [6] "sex=male ; total length=136 mm; tail length=66 mm; hind foot with claw=70 mm; ear from notch=15 mm; weight=13.5 g"                                                                                                                  
 [7] ""                                                                                                                                                                                                                                   
 [8] ""                                                                                                                                                                                                                                   
 [9] "ear from notch=14 mm; hind foot with claw=21 mm; tail length=66 mm; total length=143 mm; verbatim collector=L. S. Miller ; verbatim collector=L. S. Miller 7-29-XII-49 ; sex=female"                                                
[10] NA

In [ ]:
vertnet$tail_length <- sapply(vertnet$dynamicproperties,function(x){numextract(str_extract(string=x, pattern=regex("(?<=tail).*",ignore_case = TRUE)))[1]})

# This will capture a bunch of entries formatted as {"measurements":"135-60-19-16", "weightInGrams":"16.2" } #
NAvalues <- is.na(vertnet$tail_length)
vertnet[NAvalues,"tail_length"] <- sapply(vertnet[NAvalues,"dynamicproperties"],function(x){numextract(str_extract(string=x, pattern=regex("(?<=\"measurements\":).*",ignore_case = TRUE)))[2]})
vertnet$tail_length <- as.numeric(as.character(vertnet$tail_length))

# Get body length data, same approach as with tail #
# 1. 
vertnet$body_length <- as.numeric(as.character(sapply(vertnet$dynamicproperties,function(x){numextract(str_extract(string=x, pattern=regex("(?<=total).*",ignore_case = TRUE)))[1]}))) - vertnet$tail_length
# 2.
NAvalues <- is.na(vertnet$body_length)
vertnet[NAvalues,"body_length"] <- as.numeric(as.character(sapply(vertnet[NAvalues,"dynamicproperties"],function(x){numextract(str_extract(string=x, pattern=regex("(?<=\"measurements\":).*",ignore_case = TRUE)))[1]}))) - vertnet[NAvalues,"tail_length"]

# Now the golden ratio #
vertnet$tail_body_ratio <- vertnet$tail_length/vertnet$body_length

## Burke -- redundant, no longer needed

In [20]:
burke <- read.csv("BurkeMammalData011818-0842.csv")

burke %<>% 
    mutate(ids=paste("UWBM","Mamm",UWBM,sep=":")) %>%
    dplyr::select(ids,Scientific.Name,Sex,State,Locality,Longitude,Latitude,Preparation,Age,Date,Total.Length,Tail.Vertical.Length,Foot.Length,Ear.Length,Weight) %>%
    set_colnames(c("ids","scientificname","sex","stateprovince","locality","decimallongitude","decimallatitude",
                   "preparations","lifestage","year","total","tail","hind_foot","ear","weight")) %>%
    mutate(hastissue=NA,habitat=NA,dynamicproperties=NA)

head(burke)   

ids,scientificname,sex,stateprovince,locality,decimallongitude,decimallatitude,preparations,lifestage,year,total,tail,hind_foot,ear,weight,hastissue,habitat,dynamicproperties
UWBM:Mamm:72501,Peromyscus maniculatus,female,Washington,Frenchman Coulee,-119.9833,47.01667,skin/skull/body skel,adult,5/13/94,167,80,20,18,23,NA,NA,NA
UWBM:Mamm:72502,Peromyscus maniculatus,male,Washington,Frenchman Coulee,-119.9833,47.01667,skin/skull/body skel,adult,5/13/94,156,73,20,16,24,NA,NA,NA
UWBM:Mamm:72503,Peromyscus maniculatus,male,Washington,Frenchman Coulee,-119.9833,47.01667,skin/skull/body skel,adult,5/13/94,150,67,21,16,20,NA,NA,NA
UWBM:Mamm:72504,Peromyscus maniculatus,female,Washington,Frenchman Coulee,-119.9833,47.01667,skin/skull/body skel,adult,5/13/94,159,68,19,17,20,NA,NA,NA
UWBM:Mamm:72505,Peromyscus maniculatus,male,Washington,Frenchman Coulee,-119.9833,47.01667,skin/skull/body skel,adult,5/13/94,155,68,20,17,23,NA,NA,NA
UWBM:Mamm:72506,Peromyscus maniculatus,female,Washington,Frenchman Coulee,-119.9833,47.01667,skin/skull/body skel,adult,5/13/94,161,70,20,19,21,NA,NA,NA
